In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale, minmax_scale, robust_scale
from scipy.special import expit
window = 288 # 1 day


In [5]:
file_name = './historic_data/BTC-USD_300_200173.csv'
# Read csv, use Time column as a datetime index, and sort by this index
df = pd.read_csv(file_name, index_col='Time', parse_dates=True, infer_datetime_format=True).sort_index()
# Resample the data to a longer time interval, keeping the OHLCV data correct
#df = df.resample('15Min').apply({'Open' : 'first', 'High' : 'max', 'Low' : 'min', 'Close' : 'last', 'Volume' : 'sum'})
# Calculate the Oracle stance
df['Stance'] = df['Close'].rolling(window=window,center=True).mean().diff().rolling(window=window,center=True).mean()
# https://stackoverflow.com/questions/31287552/logarithmic-returns-in-pandas-dataframe
#df['CloseGrossReturn'] = df['Close'].pct_change()
df['CloseLogReturn'] = np.log(df['Close'] / df['Close'].shift(1))
# Scale a column to have variance of 1, do not shift the mean
#df['CloseReturnVNorm'] = scale(df['CloseLogReturn'].values, with_mean=False)
#df['CloseReturnMMNorm'] = minmax_scale(df['CloseLogReturn'].values, feature_range=(0, 1))
#df['CloseReturnRNorm'] = robust_scale(df['CloseLogReturn'].values, with_centering=False)

#df['VolumeMMNorm'] = minmax_scale(df['Volume'].values, feature_range=(0, 1))
#df['StanceVNorm'] = scale(df['Stance'].values, with_mean=False)
#df['StanceMMNorm'] = minmax_scale(df['Stance'].values, feature_range=(0, 1))

df['StanceTanh'] = np.tanh(df['Stance'])
df['VolumnSigm'] = expit(df['Volume'])

# Create categorical columns from some aspect of the time
df = pd.concat([df, pd.get_dummies(df.index.weekday, prefix='DayOfWeek').set_index(df.index)], axis=1)
df = pd.concat([df, pd.get_dummies(df.index.hour, prefix='HourOfDay').set_index(df.index)], axis=1)
#df.dropna(inplace=True)
#df[-7000:-6000].plot(y=['StanceMMNorm', 'CloseReturnMMNorm', 'VolumeMMNorm'], secondary_y=['CloseReturnMMNorm'], figsize=(15, 5), grid=True)
#df[-6500:-6000].plot(y=['StanceVNorm', 'CloseReturnRNorm', 'VolumeMMNorm'],figsize=(15, 5), grid=True)

df.describe()

,Low,High,Open,Close,Volume,Stance,CloseLogReturn,StanceTanh,VolumnTanh,DayOfWeek_0,...,HourOfDay_14,HourOfDay_15,HourOfDay_16,HourOfDay_17,HourOfDay_18,HourOfDay_19,HourOfDay_20,HourOfDay_21,HourOfDay_22,HourOfDay_23
count,200173.000000,200173.000000,200173.000000,200173.000000,200173.000000,199598.000000,200172.000000,199598.000000,200173.000000,200173.000000,...,200173.000000,200173.000000,200173.000000,200173.000000,200173.000000,200173.000000,200173.000000,200173.000000,200173.000000,200173.000000
mean,5751.975625,5769.119069,5760.906018,5760.925925,48.578258,0.027821,0.000010,0.031063,0.994118,0.142352,...,0.041754,0.041789,0.041814,0.041789,0.041839,0.041704,0.041629,0.041649,0.041679,0.041729
std,3869.464324,3887.074864,3878.595236,3878.548711,78.463278,1.140054,0.003223,0.489230,0.033663,0.349412,...,0.200027,0.200107,0.200164,0.200107,0.200221,0.199912,0.199740,0.199786,0.199855,0.199970
min,0.060000,748.440000,0.060000,746.610000,0.000001,-8.424427,-0.210212,-1.000000,0.500000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2359.610000,2369.000000,2364.120000,2364.110000,12.029536,-0.222282,-0.000751,-0.218692,0.999994,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6305.280000,6315.000000,6309.860000,6309.900000,25.426737,0.036209,0.000000,0.036194,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7853.000000,7880.450000,7866.720000,7866.720000,54.035212,0.316492,0.000858,0.306331,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,19890.990000,19891.990000,19891.990000,19891.980000,2514.557264,10.149591,0.102285,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [20]:
df.head()

,Low,High,Open,Close,Volume,Stance,CloseGrossReturn,CloseLogReturn,CloseReturnVNorm,CloseReturnMMNorm,...,HourOfDay_14,HourOfDay_15,HourOfDay_16,HourOfDay_17,HourOfDay_18,HourOfDay_19,HourOfDay_20,HourOfDay_21,HourOfDay_22,HourOfDay_23
Time,,,,,,,,,,,,,,,,,,,,,
2016-12-16 09:00:00,779.92,780.36,780.24,780.34,1.806340,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2016-12-16 09:05:00,779.95,780.82,779.95,780.82,5.345637,NaN,0.000615,0.000615,0.190808,0.674653,...,0,0,0,0,0,0,0,0,0,0
2016-12-16 09:10:00,780.51,780.82,780.82,780.82,2.675360,NaN,0.000000,0.000000,0.000000,0.672685,...,0,0,0,0,0,0,0,0,0,0
2016-12-16 09:15:00,780.53,780.82,780.82,780.76,5.917650,NaN,-0.000077,-0.000077,-0.023845,0.672439,...,0,0,0,0,0,0,0,0,0,0
2016-12-16 09:20:00,780.54,780.82,780.54,780.82,2.100960,NaN,0.000077,0.000077,0.023845,0.672931,...,0,0,0,0,0,0,0,0,0,0
